<a href="https://colab.research.google.com/github/owemecoffee/ai-learn/blob/main/CGAN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CGAN Lab
This notebook follows [A PyTorch CGAN Tutorial](https://learnopencv.com/deep-convolutional-gan-in-pytorch-and-tensorflow/) and aims at understand CGAN process. More will be covered below.

TODO:


*   显示每一次生成器生成的图片
*   按循环顺序给出每一轮的损失



## Process Fake Dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import argparse
import numpy as np
import matplotlib.pyplot as plt


### Load Dataset

In [2]:
#load dataset, keep image size at (3,64,64) and 2 classes.
train_dataset = torchvision.datasets.FakeData(
    image_size = (3,64,64)
    ,num_classes =2
    ,transform=transforms.Compose([
        transforms.ToTensor() #turn to tensor
    ])
)

In [26]:
print("The dataset has :",len(train_dataset)," items.")
print("With those ",np.unique(train_dataset.num_classes)," targets.")

#show one sample
sample = next(iter(train_dataset))
image, label = sample
plt.imshow(image)
print("Label : ", label)

The dataset has : 1000  items.
With those  [2]  targets.
Label :  0




```
torchvision.transforms.Compose
```
To composes several transforms together. 



### Preprocessing
Use normalization. Mapping pixel values between [-1, 1] have proven to be useful while training GANs.

In [3]:
#process images, compose 
train_transform = transforms.Compose([
    transforms.ToTensor(),
    #RGB images has 3 values each for  using the mean and standard deviation
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=10, shuffle=True)

### Weight Initialization
*   Used for generator and distriminator.
*   Convolution-layer weights are initialized from a zero-centered normal distribution, with a standard deviation of 0.02.

* batch-normalization layer weights are initialized with a normal distribution, having mean 1 and a standard deviation of 0.02. The bias is initialized with zeros.

In [4]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)    

## Generator
The generator is a fully-convolutional network that inputs a noise vector (latent_dim) to output an image of 3 x 64 x 64. 

In [5]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Block 1:input is Z, going into a convolution
            nn.ConvTranspose2d(3, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            # Block 2: input is (64 * 8) x 4 x 4
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            # Block 3: input is (64 * 4) x 8 x 8
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            # Block 4: input is (64 * 2) x 16 x 16
            nn.ConvTranspose2d(64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # Block 5: input is (64) x 32 x 32
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: output is (3) x 64 x 64
        )

    def forward(self, input):
        output = self.main(input)
        return output    

## Discriminator

设计的理由？



```
torch.nn.Conv2d
```
parameters: in_channel, out_channel, kernel size, stride, padding, bias





```
LeakyReLU()
```



In [6]:
# Discriminator Model Class Definition
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Block 1: input is (3) x 64 x 64
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 2: input is (64) x 32 x 32
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 3: input is (64*2) x 16 x 16
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 4: input is (64*4) x 8 x 8
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 5: input is (64*8) x 4 x 4
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
            nn.Flatten()
            # Output: 1
        )

    #fed an image, returns the output 1 (the image is real) or 0 (it is fake).
    def forward(self, input):
        output = self.main(input)
        return output


In [7]:
device = torch.device('cuda')

#the generator and discriminator models are moved to GPU and initializes all the parametric layers.
generator = Generator().to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
    (13): Flatten(start_dim=1, end_d

In [8]:
adversarial_loss = nn.BCELoss() 

In [9]:
def generator_loss(fake_output, label):
    gen_loss = adversarial_loss(fake_output, label)
    print(gen_loss)
    return gen_loss

In [10]:
def discriminator_loss(output, label):
    disc_loss = adversarial_loss(output, label)
    print(disc_loss)
    return disc_loss

### Adam Optimizer


In [11]:
learning_rate = 0.0002 
G_optimizer = optim.Adam(generator.parameters(), lr = learning_rate, betas=(0.5, 0.999))
D_optimizer = optim.Adam(discriminator.parameters(), lr = learning_rate, betas=(0.5, 0.999))

## Training



```
torch.randn
```
Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1. 

parameter: size


## Training Discriminator

In [13]:
for epoch in range(1, 100): 
    D_loss_list, G_loss_list = [], []
   
    for index, (real_images, _) in enumerate(train_loader):
      #enumerate: count list with index
      D_optimizer.zero_grad()
      real_images = real_images.to(device)
      
      real_target = Variable(torch.ones(real_images.size(0)).to(device))
      fake_target = Variable(torch.zeros(real_images.size(0)).to(device))

      #pass the real images through a discriminator, calculate the loss D_real_loss, 
      #and then backpropagate it through the discriminator network.
      output = discriminator(real_images)
      real_target = torch.reshape(real_target,(10,1))
      D_real_loss = discriminator_loss(output, real_target)
      D_real_loss.backward()
 
      noise_vector = torch.randn(real_images.size(0), 3, 1, 1, device=device)  
      noise_vector = noise_vector.to(device)

      #randomy produce noise tensor to generate images.
      generated_image = generator(noise_vector)
      output = discriminator(generated_image.detach())
      fake_target = torch.reshape(fake_target,(10,1))
      D_fake_loss = discriminator_loss(output,fake_target)

      # train discriminator with fake loss
      D_fake_loss.backward()
      
      D_total_loss = D_real_loss + D_fake_loss
      D_loss_list.append(D_total_loss)
      
      # updates the discriminator parameters, 
      D_optimizer.step()


## Training Generator

In [14]:
# Train G on D's output
G_optimizer.zero_grad()
gen_output = discriminator(generated_image)
G_loss = generator_loss(gen_output, real_target)
G_loss_list.append(G_loss)
G_loss.backward()
G_optimizer.step()